# Usage Example - MobileNet V2

This notebook documents how to use the `tfhub-exporter`script to export and use an optimized version of the MobileNet classifier available at https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/3.

First visit https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/3 and grab the module url, e.g. by using the `Copy URL` button.

In [1]:
MODULE_URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/3'

## 1. Inspect the module inputs and outputs
The following command downloads the module data and displays all available inputs and outputs with the corresponding details.

In [2]:
!python main.py show-info $MODULE_URL

I0521 16:35:10.462617 4347246016 main.py:47] Loading module "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/3"
I0521 16:35:10.477555 4347246016 resolver.py:79] Using /var/folders/nw/pc9xb0fx7m99qv22qgl8sm6c0000gn/T/tfhub_modules to cache modules.
I0521 16:35:19.260080 4347246016 main.py:24] <Took 8.80s>

 Model Inputs
╭────────┬─────────────────────┬─────────╮
│  name  │        shape        │  dtype  │
├────────┼─────────────────────┼─────────┤
│ images │ (None, 224, 224, 3) │ float32 │
╰────────┴─────────────────────┴─────────╯

 Supported Outputs
╭───────────┬──────────────┬─────────╮
│ signature │    shape     │  dtype  │
├───────────┼──────────────┼─────────┤
│  default  │ (None, 1001) │ float32 │
╰───────────┴──────────────┴─────────╯



## 2. Export the module

The next command loads the MobileNet model, freezes the network weights and applies a number of optimizations in order to reduce the file size as well as model loading time. The final model is saved in `./mobilenetV2/`.

In [3]:
EXPORT_TARGET_DIR = './mobilenetV2'

We have to delete the target directory first in case it already exists.

In [4]:
!rm -rf $EXPORT_TARGET_DIR

In [5]:
!python main.py export $MODULE_URL $EXPORT_TARGET_DIR

I0521 16:37:02.648819 4591490496 main.py:89] Loading module "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/3"
I0521 16:37:02.651168 4591490496 resolver.py:79] Using /var/folders/nw/pc9xb0fx7m99qv22qgl8sm6c0000gn/T/tfhub_modules to cache modules.
I0521 16:37:10.420182 4591490496 main.py:24] <Took 7.77s>
I0521 16:37:10.420423 4591490496 main.py:97] Detected inputs "{'images': <tf.Tensor 'images:0' shape=(?, 224, 224, 3) dtype=float32>}"
I0521 16:37:10.420545 4591490496 main.py:98] Output "module_apply_default/MobilenetV2/Logits/output:0": "{'default': <hub.ParsedTensorInfo shape=(?, 1001) dtype=float32 is_sparse=False>}"
I0521 16:37:10.420630 4591490496 main.py:102] Normalized output name: "module_apply_default/MobilenetV2/Logits/output"
I0521 16:37:12.957731 4591490496 main.py:110] Exporting TF Hub module to "/var/folders/nw/pc9xb0fx7m99qv22qgl8sm6c0000gn/T/tmpfa7rdsv_/./mobilenetV2"
I0521 16:37:18.612892 4591490496 main.py:24] <Took 5.65s>
I0521 16:37:18.613455 

## 3. Run the exported and optimized model

Next, we want to load the optimized model and find out what the following image is classified as.
![Test Image](example.jpg)

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.saved_model import tag_constants

with tf.Session(graph=tf.Graph()) as sess:
    metagraph = tf.saved_model.loader.load(sess, [tag_constants.SERVING], EXPORT_TARGET_DIR)
    
    input_mappings = metagraph.signature_def['serving_default'].inputs
    output_mappings = metagraph.signature_def['serving_default'].outputs
    
    # We know the input names from either the 'show-info' command, or the 'Module Inputs' information
    # displayed at the end of the 'export' command
    input_images = sess.graph.get_tensor_by_name(input_mappings['images'].name)
    
    # The output tensor information is always exported under the name 'output'
    output = sess.graph.get_tensor_by_name(output_mappings['output'].name)
    
    # Load the example image the tensorflow way
    file_reader = tf.read_file('example.jpg')
    example = tf.image.decode_jpeg(file_reader)
    images = tf.expand_dims(example, 0)
    images = tf.image.resize_image_with_crop_or_pad(images, 224, 224)
    
    # See https://www.tensorflow.org/hub/common_signatures/images#input
    images = tf.image.convert_image_dtype(images, tf.float32)

    # Load image
    images = sess.run(images)
    
    # Run classifier on image
    result = sess.run(output, feed_dict={
        input_images: images
    })
    
    # Find class with highest probability
    classifications = result[0]
    most_probable_class = np.argmax(classifications)
    print(f'Most probable class: {most_probable_class}')

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Instructions for updating:
Colocations handled automatically by placer.
Most probable class: 152


So the most probable class is **152**. Lets find out what the corresponding label is.

In [7]:
import urllib.request

data = urllib.request.urlopen('https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
lines = list([str(line) for line in data])

print(lines[most_probable_class])

b'Chihuahua\n'


Yep, **Chihuahua** sounds about right. Looks like our exported model is working as expected.